In [1]:
import requests
from bs4 import BeautifulSoup

main_cite = "http://www.yrcc.gov.cn/hhyl/hhjs/"  # 黄河记事的主要站点
soup = BeautifulSoup(requests.get(main_cite).content.decode('utf-8'), 'html.parser')  # 主要站点的内容

In [2]:
tags = soup.select(".hhjsM")  # 所有表格的标签
time_dic = {}
for tag in tags:
    name = tag.select('.hhjsTit')[0].text  # 时期
    time = tag.find_all('a')[0].get('href').strip(".").strip(r"/")  # 子链接
    time_dic[time] = name
    
time_dic

{'yxcqzg': '虞夏至春秋战国时期',
 'qhsq': '秦汉时期',
 'wjnbc': '魏晋南北朝时期',
 'stwd': '隋唐五代时期',
 'bs': '北宋时期',
 'jy': '金元时期',
 'md': '明代',
 'qd': '清代',
 'mgsq': '民国时期',
 'zhrmghg': '中华人民共和国时期'}

In [ ]:
def clean_event(event):
    pass

In [23]:
import pandas as pd
df = pd.DataFrame()

for time in time_dic.keys():

    time_cite = "".join([main_cite, time])
    time_soup = BeautifulSoup(requests.get(time_cite).content.decode('utf-8'), 'html.parser')  # 某个时期的子站点

    tags = time_soup.find("table").find_all("a")
    for tag in tags:
        href = tag.get('href').strip(".")
        name = tag.text
        html_cite = "".join([time_cite, href])  # 某个具体事件的网页
        
        if href.endswith("html"):
            print("Scraping data in page: {}".format(name), "\n", html_cite)
            
            event_soup = BeautifulSoup(requests.get(html_cite).content.decode('utf-8'), 'html.parser')  # 具体某个大事件
            contents = event_soup.find("table").find_all("p")  # 选取段落标签
            
            texts = []  # 一个空的列表，接收所有相关的文本
            title = name  # 默认的题目是这个页面的名字
            period = time_dic[time]  # 时段是当前循环的时段
            
            for i, content in enumerate(contents):
                if content.find("strong"):  # 如果这个是一个加粗的内容，这就是题目
                    # 将上一个存储的事件存储，如果不为空
                    if len(texts) > 0:
                        event = "".join(texts)
                        df = df.append({
                            'period': period,
                            'title': title,
                            'event': event
                        }, ignore_index=True)

                    title = content.text  # 这个事件的题目
                    texts = []  # 新建一个空的列表存储这个事件的文字
                else:
                    texts.append(content.text)
            event = "".join(texts)

            df = df.append({
                            'period': period,
                            'title': title,
                            'event': event
                        }, ignore_index=True)

df          

Scraping data in page: 大禹治水 
 http://www.yrcc.gov.cn/hhyl/hhjs/yxcqzg/201108/t20110812_89636.html
Scraping data in page: 商侯冥治河 
 http://www.yrcc.gov.cn/hhyl/hhjs/yxcqzg/201108/t20110812_89608.html
Scraping data in page: 商都数迁 
 http://www.yrcc.gov.cn/hhyl/hhjs/yxcqzg/201108/t20110812_89606.html
Scraping data in page: 陕西岐山地震 
 http://www.yrcc.gov.cn/hhyl/hhjs/yxcqzg/201108/t20110812_89599.html
Scraping data in page: 葵丘之会 
 http://www.yrcc.gov.cn/hhyl/hhjs/yxcqzg/201108/t20110812_89595.html
Scraping data in page: 晋、秦泛舟之役 
 http://www.yrcc.gov.cn/hhyl/hhjs/yxcqzg/201108/t20110812_89583.html
Scraping data in page: 蒲津建桥 
 http://www.yrcc.gov.cn/hhyl/hhjs/yxcqzg/201108/t20110812_89582.html
Scraping data in page: 禹河大徙 
 http://www.yrcc.gov.cn/hhyl/hhjs/yxcqzg/201108/t20110812_89569.html
Scraping data in page: 山西开智伯渠灌田 
 http://www.yrcc.gov.cn/hhyl/hhjs/yxcqzg/201108/t20110812_89567.html
Scraping data in page: 引漳十二渠 
 http://www.yrcc.gov.cn/hhyl/hhjs/yxcqzg/201108/t20110812_89555.html
Scraping 

,event,period,title
0,传说时代（约公元前21世纪初） 传说中的尧舜时代，黄河流域发生大洪水，为制止洪水泛滥，尧...,虞夏至春秋战国时期,大禹治水
1,夏代（约夏前期） 据《竹书纪年》记载，夏少康帝十一年，命商侯冥主持治河。冥忠于职守，兢兢...,虞夏至春秋战国时期,商侯冥治河
2,商代（公元前16世纪～前11世纪） 据《通鉴纲目》记载：商汤元年都城在亳，后因河患数迁其...,虞夏至春秋战国时期,商都数迁
3,周幽王二年（公元前780年） 据《国语·周语上》记载：周幽王二年，发生地震，黄河三条支流...,虞夏至春秋战国时期,陕西岐山地震
4,齐桓公三十五年（公元前651年） 春秋时期，防御黄河洪水的堤防已较为普遍。据《史记·齐世...,虞夏至春秋战国时期,葵丘之会
...,...,...,...
2364,1月5日 中纪委驻水利部纪检组组长董力在黄委纪检组组长李春安等陪同下，到三门峡枢纽检查指...,中华人民共和国时期,2011年
2365,1月5日 黄河宁夏河段开始封河，封河位置位于青铜峡坝上，封河长度2千米，较2011年晚6...,中华人民共和国时期,2012年
2366,1月8日 山东省副省长贾万志一行先后到济南天桥泺口险工、历城后张险工察看黄河淌凌情况，并...,中华人民共和国时期,2013年
2367,1月 《黄河河口综合治理规划环境影响报告书》通过了水利部水规总院在北京主持的专题审查，并...,中华人民共和国时期,2014年


In [24]:
df.to_csv("黄河大事记.csv", index=False, encoding='utf-8')